In [1]:
'''
Deep Learning Brasil (www.deeplearningbrasil.com.br e https://github.com/deeplearningbrasil)
Exemplo de aproximacao de funcao usando GAN - Generative Adversarial Networks - TensorFlow.

Baseado no artigo original de Ian Goodfellow et. al.: https://arxiv.org/abs/1406.2661 e nos posts de Eric Jang: http://blog.evjang.com/2016/06/generative-adversarial-nets-in.html e Arthur Juliani https://medium.com/@awjuliani/generative-adversarial-networks-explained-with-a-classic-spongebob-squarepants-episode-54deab2fce39#.3gu2q43ah

A tecnica de discriminante em minibatch foi obtida de Tim Salimans et. al.: https://arxiv.org/abs/1606.03498.
'''
from __future__ import absolute_import
from __future__ import print_function
from __future__ import unicode_literals
from __future__ import division

import argparse
import numpy as np
from scipy.stats import norm
import tensorflow as tf
import matplotlib.pyplot as plt
from matplotlib import animation
import seaborn as sns

sns.set(color_codes=True)

seed = 42
np.random.seed(seed)
tf.set_random_seed(seed)

In [2]:
class DistribuicaoDados(object):
    def __init__(self):
        self.media = 4
        self.desvio_padrao = 0.5

    def sample(self, N):
        amostras = np.random.normal(self.media, self.desvio_padrao, N)
        amostras.sort()
        return amostras

In [3]:
class DistribuicaoDadosGerador(object):
    def __init__(self, range):
        self.range = range

    def sample(self, N):
        return np.linspace(-self.range, self.range, N) + \
            np.random.random(N) * 0.01

In [4]:
def linear(input, output_dim, scope=None, stddev=1.0):
    norm = tf.random_normal_initializer(stddev=stddev)
    const = tf.constant_initializer(0.0)
    with tf.variable_scope(scope or 'linear'):
        w = tf.get_variable('w', [input.get_shape()[1], output_dim], initializer=norm)
        b = tf.get_variable('b', [output_dim], initializer=const)
        return tf.matmul(input, w) + b

In [5]:
def gerador(input, hidden_size):
    passo1 = linear(input,hidden_size,'g0')
    passo2 = tf.nn.softplus(passo1)
    passo3 = linear(passo2, 1, 'g1')
    return passo3 

In [6]:
def discriminador(input, h_dim, minibatch_layer=True):
    passo1_h0 = linear(input,h_dim*2,'d0')
    passo2_h0 = tf.tanh(passo1_h0)
    passo1_h1 = linear(passo2_h0,h_dim*2,'d1')
    passo2_h1 = tf.tanh(passo1_h1)

    #Sem a camada de minibatch, o discriminador precisa de uma camada adicional 
    #para ter capacidade de separar as duas distribuicoes
    if minibatch_layer:
        passo3_h2 = minibatch(passo2_h1)
    else:
        passo3_h2 = tf.tanh(linear(passo2_h1, h_dim * 2, scope='d2'))

    h3 = tf.sigmoid(linear(passo3_h2, 1, scope='d3'))
    return h3

In [7]:
def otimizador(erro, var_list, taxa_aprendizado_inicial):
    decaimento = 0.95
    qtd_passos_decaimento = 150
    batch = tf.Variable(0)
    learning_rate = tf.train.exponential_decay(
        taxa_aprendizado_inicial,
        batch,
        qtd_passos_decaimento,
        decaimento,
        staircase=True
    )
    otimizador = tf.train.GradientDescentOptimizer(learning_rate).minimize(
        erro,
        global_step=batch,
        var_list=var_list
    )
    return otimizador

In [8]:
data = DistribuicaoDados()

In [9]:
gen = DistribuicaoDadosGerador(range=8)

In [10]:
num_steps = 1200
batch_size = 12
minibatch = False
log_every = 10
mlp_hidden_size = 4
#anim_path = anim_path
anim_frames = []
learning_rate = 0.03

In [11]:
#  Cria a rede pre-treinada D_pre utilizando maximum likehood. A rede e 
# utilizada para repassar informacao do gradiente para a rede geradora 
# no primeiro passo.
with tf.variable_scope('D_pre'):
    pre_input = tf.placeholder(tf.float32, shape=(batch_size, 1))
    pre_labels = tf.placeholder(tf.float32, shape=(batch_size, 1))
    D_pre = discriminador(pre_input, mlp_hidden_size, minibatch)
    pre_loss = tf.reduce_mean(tf.square(D_pre - pre_labels))
    pre_opt = otimizador(pre_loss, None, learning_rate)

In [12]:
# Define a rede geradora - esta obtem dados de entrada a partir de uma 
# distribuicao com ruidos, e repassa para uma rede MLP.
with tf.variable_scope('Gen'):
    z = tf.placeholder(tf.float32, shape=(batch_size, 1))
    G = gerador(z, mlp_hidden_size)

In [13]:
# O discriminador tentara diferenciar as amostras reais (x) 
# das sinteticas (z)
with tf.variable_scope('Disc') as scope:
    x = tf.placeholder(tf.float32, shape=(batch_size, 1))
    D1 = discriminador(x, mlp_hidden_size, minibatch)
    scope.reuse_variables()
    #cria uma segunda rede de discriminacao, pois no tensorflow nao seria 
    #possivel uma rede com duas entradas diferentes
    D2 = discriminador(G, mlp_hidden_size, minibatch)

In [14]:
# Define a funcao de erro, para a rede de discriminacao e de geracao
loss_d = tf.reduce_mean(-tf.log(D1) - tf.log(1 - D2))
loss_g = tf.reduce_mean(-tf.log(D2))

d_pre_params = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope='D_pre')
d_params = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope='Disc')
g_params = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope='Gen')

#define o otimizador para ambas as redes
opt_d = otimizador(loss_d, d_params, learning_rate)
opt_g = otimizador(loss_g, g_params, learning_rate)

In [18]:
init = tf.global_variables_initializer()
session = tf.Session()
session.run(init)

In [19]:
log_loss_d = []
log_loss_g = []
# discriminador pre-treinado
num_pretrain_steps = 1000

In [20]:
for step in range(num_pretrain_steps):
    d = (np.random.random(batch_size) - 0.5) * 10.0
    labels = norm.pdf(d, loc=data.media, scale=data.desvio_padrao)
    pretrain_loss, _ = session.run([pre_loss, pre_opt], {
        pre_input: np.reshape(d, (batch_size, 1)),
        pre_labels: np.reshape(labels, (batch_size, 1))
    })

In [21]:
weightsD = session.run(d_pre_params)

In [22]:
# copia os pesos da rede a partir da rede pre-treinada para a rede discriminativa D
for i, v in enumerate(d_params):
    session.run(v.assign(weightsD[i]))

### Teste

In [42]:
# atualiza o discriminador
x = data.sample(batch_size)
z = gen.sample(batch_size)

In [43]:
x

array([ 3.2883316 ,  3.30082146,  3.5032832 ,  3.58183075,  3.63778012,
        3.98024173,  4.06537193,  4.14472554,  4.2602804 ,  4.27052549,
        4.41461179,  4.49261009])

In [45]:
z

array([-7.99375158, -6.53948529, -5.08931483, -3.63001797, -2.17751654,
       -0.72708681,  0.73267037,  2.19171085,  3.63986103,  5.10056468,
        6.5464166 ,  8.0073245 ])

In [41]:
loss_d, _ = session.run([loss_d, opt_d], {
    x: np.reshape(x, (batch_size, 1)),
    z: np.reshape(z, (batch_size, 1))
})

TypeError: unhashable type: 'numpy.ndarray'